# BRENT CRUDE OIL PRICE FORECASTING (SARIMA and LSTM models)

In [6]:
# Required modules

# For common operations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose # For time series decomposition
from pmdarima import auto_arima


# For LSTM model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import load_model  # Allows load a previously saved model.

# To evaluate the models
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# To enable interactive plots

## For Jupyter web (requires ipympl module)
#%matplotlib widget

## For IDEs, like PyCharm
import matplotlib
matplotlib.use('nbagg')


### Data loading

In [7]:
full_data = pd.read_csv('D:/datasets/brent_daily_prices.csv', parse_dates=['DATE'], date_format='%d/%m/%Y', index_col='DATE', na_values='.')  # In the original time series, NA values are represented by a period (.)
data = full_data['DCOILBRENTEU']
data.rename_axis('date', inplace=True)
data.rename('brent_crude_oil', inplace=True)
data.fillna(method='ffill', inplace=True)  # Replace NaN values with the last valid observation.
data.index = pd.DatetimeIndex(data.index).to_period('B').to_timestamp()  # Sets the frequency for the time series.
data

date
1987-05-20    18.63
1987-05-21    18.45
1987-05-22    18.55
1987-05-25    18.60
1987-05-26    18.63
              ...  
2023-08-01    85.34
2023-08-02    84.01
2023-08-03    86.19
2023-08-04    87.38
2023-08-07    86.47
Freq: B, Name: brent_crude_oil, Length: 9449, dtype: float64

### Exploratory Data Analysis

In [8]:
# Basic EDA of the data
print(data.describe())
print('\n')
print(data.info())
print('Missing values: ', data.isna().sum())

# Plots the data
plt.figure(figsize=(10, 4))
plt.plot(data)
plt.title('Brent Crude Oil Price since 1987')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.show()

count    9449.000000
mean       49.155899
std        32.956536
min         9.100000
25%        19.150000
50%        40.800000
75%        72.050000
max       143.950000
Name: brent_crude_oil, dtype: float64


<class 'pandas.core.series.Series'>
DatetimeIndex: 9449 entries, 1987-05-20 to 2023-08-07
Freq: B
Series name: brent_crude_oil
Non-Null Count  Dtype  
--------------  -----  
9449 non-null   float64
dtypes: float64(1)
memory usage: 147.6 KB
None
Missing values:  0


<IPython.core.display.Javascript object>

In [9]:
# Checking the composition of the data
seasonal_decompose(data).plot()
plt.xticks(rotation=45)
plt.show()

<IPython.core.display.Javascript object>

### Data splitting

In [10]:
train_data = data.iloc[:len(data) - 30]
test_data = data.iloc[len(data) - 30:]

## SARIMA model

### Getting the parameters for the model

In [11]:
# Runs auto_arima function to get the parameters for the SARIMA model
opt_model = auto_arima(train_data, maxiter=100, trace=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=30304.148, Time=8.16 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=30327.930, Time=0.60 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=30317.381, Time=1.15 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=30316.773, Time=1.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=30326.151, Time=0.45 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=30314.723, Time=5.70 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=30314.425, Time=3.73 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=30303.250, Time=6.64 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=30310.245, Time=3.94 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=30320.821, Time=13.08 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=30305.257, Time=8.82 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=30303.251, Time=6.69 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=30308.252, Time=3.55 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=30307.202, Time=4.77 sec
 ARIMA(3,1,2)(0,0,

### Model training

In [12]:
# Trains the model according to the auto_arima output -> (2,1,3)x(0,0,0,0)
sarima_model_eval = SARIMAX(train_data, order=(2, 1, 3), seasonal_order=(0, 0, 0, 0))
estimator_eval = sarima_model_eval.fit()

# Gets forecast for evaluation
preds = estimator_eval.forecast(len(test_data))

# Plot the results
test_data.plot(color='blue', label='Actual')
preds.plot(color='green', label='Forecasts (30 days)')

plt.title('Brent Crude Oil Price Forecast (SARIMA model evaluation)')
plt.xlabel('Date')
plt.ylabel('Oil price (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### Model evaluation

In [13]:
rmse = np.sqrt(mean_squared_error(test_data.values, preds.values))
mae = mean_absolute_error(test_data.values, preds.values)
mape = mean_absolute_percentage_error(test_data.values, preds.values)

print('Root Mean Square Error (RMSE): {} \nMean Absolute Error (MAE): {} \nMean Absolute Percentage Error (MAPE): {}'. format(np.round(rmse, 3), np.round(mae, 3), np.round(mape, 3)))

Root Mean Square Error (RMSE): 7.543 
Mean Absolute Error (MAE): 6.431 
Mean Absolute Percentage Error (MAPE): 0.078


### Using the model to forecasts Brent crude oil price for the following 15 days

In [15]:
# Sets the model
sarima_model_forecast = SARIMAX(data, order=(2,1,3), seasonal_order=(0,0,0,0))
estimator_forecast = sarima_model_forecast.fit()

# Makes predictions
steps_ahead = 15
forecasts = estimator_forecast.forecast(steps_ahead)
ci = estimator_forecast.conf_int()

# Displays the results
short_data = data[data.index.year >= 2023]
short_data.plot(color='blue', label='Actual')
forecasts.plot(color='red', label='Forecasts')

plt.title('Brent Crude Oil Price Forecast with SARIMA (15 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

print('Forecasts for the following {} days: \n'.format(steps_ahead))
print(forecasts)

<IPython.core.display.Javascript object>

Forecasts for the following 15 days: 

2023-08-08    86.307381
2023-08-09    86.385513
2023-08-10    86.422531
2023-08-11    86.357439
2023-08-14    86.391178
2023-08-15    86.394936
2023-08-16    86.376384
2023-08-17    86.389143
2023-08-18    86.387758
2023-08-21    86.382935
2023-08-22    86.387345
2023-08-23    86.386131
2023-08-24    86.385039
2023-08-25    86.386454
2023-08-28    86.385853
Freq: B, Name: predicted_mean, dtype: float64


## LSTM model

### Data preparation

In [16]:
# Reshapes the data to feed the model
full_data_lstm = data.values.reshape(-1, 1)
train_data_lstm = train_data.values.reshape(-1, 1)
test_data_lstm = test_data.values.reshape(-1, 1)

# Defines train and test sets
X_train = []
y_train = []
ws = 30 # Window size: indicates the number of previous time steps. The more, may lead to higher accuracy, but increases complexity and training time.

for i in range(ws, len(train_data_lstm)):
    X_train.append(train_data_lstm[i - ws: i])
    y_train.append(train_data_lstm[i])

X_train, y_train = np.array(X_train), np.array(y_train)

### Model training
The model hyperparameters were chosen after evaluating many different combinations.

In [33]:
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape = (X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')  
model.fit(X_train, y_train, epochs=100, batch_size=600)

Epoch 1/100
16/16 [==============================] - 6s 253ms/step - loss: 2375.4519
Epoch 2/100
16/16 [==============================] - 4s 260ms/step - loss: 660.6410
Epoch 3/100
16/16 [==============================] - 4s 246ms/step - loss: 108.0478
Epoch 4/100
16/16 [==============================] - 4s 234ms/step - loss: 36.9575
Epoch 5/100
16/16 [==============================] - 4s 231ms/step - loss: 19.4714
Epoch 6/100
16/16 [==============================] - 4s 238ms/step - loss: 13.6139
Epoch 7/100
16/16 [==============================] - 4s 228ms/step - loss: 11.4636
Epoch 8/100
16/16 [==============================] - 4s 234ms/step - loss: 11.0681
Epoch 9/100
16/16 [==============================] - 4s 235ms/step - loss: 10.5807
Epoch 10/100
16/16 [==============================] - 4s 229ms/step - loss: 8.5157
Epoch 11/100
16/16 [==============================] - 4s 239ms/step - loss: 6.6388
Epoch 12/100
16/16 [==============================] - 4s 236ms/step - loss: 6.7650


### Plotting loss
This is useful to check if the number of epochs is adequate: 
- A flat trend at the end of the curve is desired.
- If the end of the curve has a downward trend, it could indicate an opportunity of improvement, requiring a larger number of epochs.
- If the end of the curve has an upward trend, it could indicate overfitting, so fewer epochs are required.

In [ ]:
plt.plot(range(len(model.history.history['loss'])), model.history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

### Saving the trained model

In [ ]:
model.save('model1')

### Loading a model
Please, load the model "brent_price_forecast_lstm_model" to evaluate it or make forecasts.

That model was trained with the following parameters:
- epochs = 100
- units = 150 (indicates the number of neurons in the LSTM layer)
- batch_size = 600
- activation = 'relu' (indicates the activation function in the LSTM layer)
- optimizer = 'adam' 
- loss = 'mse'
- The data to feed the model was prepared with a window size of 30.

Neural network algorithms are stochastic, which means they make use of randomness, such as initializing to random weights, and in turn the same network, with the same hyperparameters, trained on the same data can produce different results [1]. This is the reason why it is necessary to load the aforementioned model.



[1] https://machinelearningmastery.com/stochastic-in-machine-learning

In [19]:
model = load_model('brent_price_forecast_lstm_model')

### Model testing

In [20]:
prediction_set = []
batch_one = train_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))

for i in range(len(test_data)):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set] # Transforms a list of arrays into a list of single float items.
predictions = pd.Series(prediction_set, index=test_data.index)


# Plots the results
test_data.plot(color='blue', label='Actual')
predictions.plot(color='green', label='Prediction')

plt.title('Brent Crude Oil Price Forecast (LSTM model evaluation)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### Model evaluation

In [21]:
rmse = np.sqrt(mean_squared_error(test_data_lstm, predictions))
mae = mean_absolute_error(test_data_lstm, predictions)
mape = mean_absolute_percentage_error(test_data_lstm, predictions)
print('Root Mean Square Error (RMSE): {} \nMean Absolute Error (MAE): {} \nMean Absolute Percentage Error (MAPE): {}'. format(np.round(rmse, 3), np.round(mae, 3), np.round(mape, 3)))

Root Mean Square Error (RMSE): 3.677 
Mean Absolute Error (MAE): 3.224 
Mean Absolute Percentage Error (MAPE): 0.041


### Using the model to forecast the Brent crude oil price for the following 30 days

In [22]:
# Makes the predictions 
prediction_set = []
batch_one = full_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))
days_to_forecast = 30

for i in range(days_to_forecast):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set]  # Transforms a list of arrays into a list of single float items.
date_range = pd.date_range(test_data.index[-1], periods=days_to_forecast, freq='B')   
forecast = pd.Series(prediction_set, index=date_range, name='Forecast')


# Displays results
short_data = data.iloc[-250:] # Last n datapoints of the original time series.

short_data.plot(color='blue', label='Actual')
forecast.plot(color='red')

plt.title('Brent Crude Oil Price Forecast with LSTM (30 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

print('Forecasts for the following {} days: '.format(days_to_forecast))
print(forecast)

<IPython.core.display.Javascript object>

Forecasts for the following 30 days: 
2023-08-07     86.838974
2023-08-08     88.119522
2023-08-09     88.610596
2023-08-10     88.780853
2023-08-11     90.181458
2023-08-14     90.257935
2023-08-15     90.509865
2023-08-16     90.808083
2023-08-17     90.853645
2023-08-18     89.893936
2023-08-21     90.618149
2023-08-22     92.039726
2023-08-23     93.088234
2023-08-24     92.772865
2023-08-25     93.825233
2023-08-28     95.939301
2023-08-29     95.710098
2023-08-30     94.572220
2023-08-31     95.415863
2023-09-01     95.783859
2023-09-04     96.019470
2023-09-05     95.710281
2023-09-06     95.871262
2023-09-07     96.260857
2023-09-08     96.980095
2023-09-11     97.948135
2023-09-12     98.919952
2023-09-13     98.961479
2023-09-14    100.141144
2023-09-15    101.966057
Freq: B, Name: Forecast, dtype: float32


This LSTM model was trained to forecast the Brent crude oil price for 30 days ahead from the last date in the time series. That means the model could produce extremely erroneous results if it is used with long time spans (variable _days_to_forecast_), for example, 60 days or more.

## Conclusions
1. Modeling crude oil prices is a complicated task due to the high variation and volatility associated with its market. However, it's necessary to do so, as oil is one of the most important energies driving the world economy and represents a crucial factor in most industries.
   
2. In the evaluation of the SARIMA model, a Root Mean Squared Error (RMSE) of 7.543 and a Mean Absolute Error (MAE) of 6.431 were obtained. The forecast graph for the next 15 days shows a horizontal line with a value of approximately 83.5. This means that the SARIMA model estimates the Brent crude oil price for the next 15 days to be around $83.5 per barrel, on average.

3. In the evaluation of the LSTM model, a Root Mean Squared Error (RMSE) of 3.677 and a Mean Absolute Error (MAE) of 3.224 were obtained. The forecast graph for the next 30 days demonstrates its capability to capture the trend and shape of the time series of actual data.

4. Considering the errors obtained in the evaluations of each model, along with their ability to capture the trend and shape of the original time series, it's evident that the LSTM model performs significantly better in forecasting the Brent crude oil price.